In [0]:
import sys
import os
import urllib
import time
import shutil
import tarfile
import tempfile

from matplotlib import gridspec
from matplotlib import pyplot as plt

import numpy as np

from PIL import Image

from distutils.version import StrictVersion

%matplotlib inline

In [0]:
import tensorflow as tf

print(tf.__version__)
if StrictVersion(tf.__version__) < StrictVersion('1.14.0'):
    raise ImportError('Please upgrade your TensorFlow installation to v1.14.*.')

In [0]:
from tensorflow.python.compiler.tensorrt import trt_convert

## Define DeepLabModel

In [0]:
class DeepLabModel(object):
    """Class to load deeplab model and run inference."""

    INPUT_TENSOR_NAME = 'ImageTensor:0'
    OUTPUT_TENSOR_NAME = 'SemanticPredictions:0'
    INPUT_SIZE = 513

    def __init__(self, model_path):
        """Creates and loads deeplab model."""
        tf.reset_default_graph()
        
        self.graph = tf.Graph()

        graph_def = None
        with tf.gfile.GFile(model_path, 'rb') as f:
            graph_def = tf.GraphDef.FromString(f.read())

        if graph_def is None:
            raise RuntimeError('Cannot load inference graph: ' + model_path)

        with self.graph.as_default():
            tf.import_graph_def(graph_def, name='')

        self.sess = tf.Session(graph=self.graph)

    def run(self, image):
        """Runs inference on a single image.

        Args:
            image: A PIL.Image object, raw input image.

        Returns:
            resized_image: RGB image resized from original input image.
            seg_map: Segmentation map of `resized_image`.
        """
        width, height = image.size
        resize_ratio = 1.0 * self.INPUT_SIZE / max(width, height)
        target_size = (int(resize_ratio * width), int(resize_ratio * height))
        resized_image = image.convert('RGB').resize(target_size, Image.ANTIALIAS)
        batch_seg_map = self.sess.run(
            self.OUTPUT_TENSOR_NAME,
            feed_dict={self.INPUT_TENSOR_NAME: [np.asarray(resized_image)]})
        seg_map = batch_seg_map[0]
        return resized_image, seg_map

    def run_benchmark(self, image, count=100):
        width, height = image.size
        resize_ratio = 1.0 * self.INPUT_SIZE / max(width, height)
        target_size = (int(resize_ratio * width), int(resize_ratio * height))
        resized_image = image.convert('RGB').resize(target_size, Image.ANTIALIAS)
        
        times = []
        for i in range(count):
            start_tm = time.time()
            self.sess.run(
                self.OUTPUT_TENSOR_NAME,
                feed_dict={self.INPUT_TENSOR_NAME: [np.asarray(resized_image)]})
            times.append(time.time() - start_tm)
    
        print('Inference : {0:.2f} ms'.format(np.array(times).mean() * 1000))

    
def create_pascal_label_colormap():
    """Creates a label colormap used in PASCAL VOC segmentation benchmark.

    Returns:
        A Colormap for visualizing segmentation results.
    """
    colormap = np.zeros((256, 3), dtype=int)
    ind = np.arange(256, dtype=int)

    for shift in reversed(range(8)):
        for channel in range(3):
            colormap[:, channel] |= ((ind >> channel) & 1) << shift
        ind >>= 3

    return colormap


def label_to_color_image(label):
    """Adds color defined by the dataset colormap to the label.

    Args:
        label: A 2D array with integer type, storing the segmentation label.

    Returns:
        result: A 2D array with floating type. The element of the array
        is the color indexed by the corresponding element in the input label
        to the PASCAL color map.

    Raises:
        ValueError: If label is not of rank 2 or its value is larger than color
        map maximum entry.
    """
    if label.ndim != 2:
        raise ValueError('Expect 2-D input label')

    colormap = create_pascal_label_colormap()

    if np.max(label) >= len(colormap):
        raise ValueError('label value too large.')

    return colormap[label]


def vis_segmentation(image, seg_map):
    """Visualizes input image, segmentation map and overlay view."""
    plt.figure(figsize=(15, 5))
    grid_spec = gridspec.GridSpec(1, 4, width_ratios=[6, 6, 6, 1])

    plt.subplot(grid_spec[0])
    plt.imshow(image)
    plt.axis('off')
    plt.title('input image')

    plt.subplot(grid_spec[1])
    seg_image = label_to_color_image(seg_map).astype(np.uint8)
    plt.imshow(seg_image)
    plt.axis('off')
    plt.title('segmentation map')

    plt.subplot(grid_spec[2])
    plt.imshow(image)
    plt.imshow(seg_image, alpha=0.7)
    plt.axis('off')
    plt.title('segmentation overlay')

    unique_labels = np.unique(seg_map)
    ax = plt.subplot(grid_spec[3])
    plt.imshow(
        FULL_COLOR_MAP[unique_labels].astype(np.uint8), interpolation='nearest')
    ax.yaxis.tick_right()
    plt.yticks(range(len(unique_labels)), LABEL_NAMES[unique_labels])
    plt.xticks([], [])
    ax.tick_params(width=0.0)
    plt.grid('off')
    plt.show()

LABEL_NAMES = np.asarray([
    'background', 'aeroplane', 'bicycle', 'bird', 'boat', 'bottle', 'bus',
    'car', 'cat', 'chair', 'cow', 'diningtable', 'dog', 'horse', 'motorbike',
    'person', 'pottedplant', 'sheep', 'sofa', 'train', 'tv'
])

FULL_LABEL_MAP = np.arange(len(LABEL_NAMES)).reshape(len(LABEL_NAMES), 1)
FULL_COLOR_MAP = label_to_color_image(FULL_LABEL_MAP)

# Download the pretrained model

Download the model configuration file and checkpoint containing pretrained weights by using the following command.

In [0]:
MODEL_NAME = 'xception_coco_voctrainaug'  # @param ['mobilenetv2_dm05_coco_voc_trainaug', 'mobilenetv2_dm05_coco_voc_trainval', 'mobilenetv2_coco_voctrainaug', 'mobilenetv2_coco_voctrainval', 'xception_coco_voctrainaug', 'xception_coco_voctrainval']

_DOWNLOAD_URL_PREFIX = 'http://download.tensorflow.org/models/'
_MODEL_URLS = {
    'mobilenetv2_dm05_coco_voc_trainaug':
        'deeplabv3_mnv2_dm05_pascal_trainaug_2018_10_01.tar.gz',
    'mobilenetv2_dm05_coco_voc_trainval':
        'deeplabv3_mnv2_dm05_pascal_trainval_2018_10_01.tar.gz',
    'mobilenetv2_coco_voctrainaug':
        'deeplabv3_mnv2_pascal_train_aug_2018_01_29.tar.gz',
    'mobilenetv2_coco_voctrainval':
        'deeplabv3_mnv2_pascal_trainval_2018_01_29.tar.gz',
    'xception_coco_voctrainaug':
        'deeplabv3_pascal_train_aug_2018_01_04.tar.gz',
    'xception_coco_voctrainval':
        'deeplabv3_pascal_trainval_2018_01_04.tar.gz',
}
_MODEL_DIRS = {
    'mobilenetv2_dm05_coco_voc_trainaug':
        'deeplabv3_mnv2_dm05_pascal_trainaug',
    'mobilenetv2_dm05_coco_voc_trainval':
        'deeplabv3_mnv2_dm05_pascal_trainval',
    'mobilenetv2_coco_voctrainaug':
        'deeplabv3_mnv2_pascal_train_aug',
    'mobilenetv2_coco_voctrainval':
        'deeplabv3_mnv2_pascal_trainval',
    'xception_coco_voctrainaug':
        'deeplabv3_pascal_train_aug',
    'xception_coco_voctrainval':
        'deeplabv3_pascal_trainval',
}
_TARBALL_NAME = 'deeplab_model.tar.gz'

MODEL_DIR = 'models'
tf.gfile.MakeDirs(MODEL_DIR)

download_path = os.path.join(MODEL_DIR, _TARBALL_NAME)
print('downloading model, this might take a while...')

urllib.request.urlretrieve(_DOWNLOAD_URL_PREFIX + _MODEL_URLS[MODEL_NAME],
                   download_path)
print('download completed.')

with tarfile.open(download_path) as tar:
    tar.extractall(MODEL_DIR)

In [0]:
MODEL_PATH = os.path.join(MODEL_DIR, _MODEL_DIRS[MODEL_NAME])
FROZEN_GRAPH_NAME = 'frozen_inference_graph.pb'

In [0]:
MODEL_NAME

In [0]:
os.path.join(MODEL_PATH, FROZEN_GRAPH_NAME)

In [0]:
deeplab_model = DeepLabModel(os.path.join(MODEL_PATH, FROZEN_GRAPH_NAME))

In [0]:
original_im = Image.open('parrot.jpg')
resized_im, seg_map = deeplab_model.run(original_im)

vis_segmentation(resized_im, seg_map)

In [0]:
deeplab_model.run_benchmark(original_im)

# Load deeplab model and convert graph 

In [0]:
tf.reset_default_graph()
graph = tf.Graph()

graph_def = None
with tf.gfile.GFile(os.path.join(MODEL_PATH, FROZEN_GRAPH_NAME), 'rb') as f:
    graph_def = tf.GraphDef.FromString(f.read())

In [0]:
output_names = ['SemanticPredictions']

converter = trt_convert.TrtGraphConverter(
    input_graph_def=graph_def,
    nodes_blacklist=output_names, #output nodes
    max_batch_size=1,
    is_dynamic_op=True,
    max_workspace_size_bytes=trt_convert.DEFAULT_TRT_MAX_WORKSPACE_SIZE_BYTES,
    precision_mode=trt_convert.TrtPrecisionMode.FP16,
    minimum_segment_size=3)

trt_graph = converter.convert()

In [0]:
trt_engine_opts = len([1 for n in trt_graph.node if str(n.op) == 'TRTEngineOp'])
print("trt_engine_opts = {}".format(trt_engine_opts))

In [0]:
with tf.gfile.GFile(os.path.join(MODEL_DIR, MODEL_NAME + '_fp16.pb'), 'wb') as f:
    f.write(trt_graph.SerializeToString())

# Load FP16 Model and Run inference.

In [0]:
deeplab_fp16_model = DeepLabModel(os.path.join(MODEL_DIR, MODEL_NAME + '_fp16.pb'))

In [0]:
original_im = Image.open('parrot.jpg')
resized_im, seg_map = deeplab_model.run(original_im)

vis_segmentation(resized_im, seg_map)

In [0]:
deeplab_model.run_benchmark(original_im)

In [0]:
INPUT_TENSOR_NAME = 'ImageTensor:0'
OUTPUT_TENSOR_NAME = 'SemanticPredictions:0'
INPUT_SIZE = 513
FROZEN_GRAPH_NAME = 'frozen_inference_graph'

In [0]:
def create_pascal_label_colormap():
    color_map = np.zeros((256, 3), dtype=int)
    ind = np.arange(256, dtype=int)

    for shift in reversed(range(8)):
        for channel in range(3):
              color_map[:, channel] |= ((ind >> channel) & 1) << shift
        ind >>= 3

    return color_map

In [0]:
def label_to_color_image(label):
    if label.ndim != 2:
        raise ValueError('Expect 2-D input label')

    color_map = create_pascal_label_colormap()

    if np.max(label) >= len(color_map):
        raise ValueError('label value too large.')

    return color_map[label]


In [0]:
def vis_segmentation(image, seg_map):
    plt.figure(figsize=(15, 5))
    grid_spec = gridspec.GridSpec(1, 4, width_ratios=[6, 6, 6, 1])

    plt.subplot(grid_spec[0])
    plt.imshow(image)
    plt.axis('off')
    plt.title('input image')

    plt.subplot(grid_spec[1])
    seg_image = label_to_color_image(seg_map).astype(np.uint8)
    plt.imshow(seg_image)
    plt.axis('off')
    plt.title('segmentation map')

    plt.subplot(grid_spec[2])
    plt.imshow(image)
    plt.imshow(seg_image, alpha=0.7)
    plt.axis('off')
    plt.title('segmentation overlay')

    unique_labels = np.unique(seg_map)
    ax = plt.subplot(grid_spec[3])
    plt.imshow(FULL_COLOR_MAP[unique_labels].astype(np.uint8), interpolation='nearest')
    ax.yaxis.tick_right()
    plt.yticks(range(len(unique_labels)), LABEL_NAMES[unique_labels])
    plt.xticks([], [])
    ax.tick_params(width=0.0)
    plt.grid('off')
    plt.show()

In [0]:
LABEL_NAMES = np.asarray([
    'background', 'aeroplane', 'bicycle', 'bird', 'boat', 'bottle', 'bus',
    'car', 'cat', 'chair', 'cow', 'diningtable', 'dog', 'horse', 'motorbike',
    'person', 'pottedplant', 'sheep', 'sofa', 'train', 'tv'
])

FULL_LABEL_MAP = np.arange(len(LABEL_NAMES)).reshape(len(LABEL_NAMES), 1)
FULL_COLOR_MAP = label_to_color_image(FULL_LABEL_MAP)

In [0]:
tf.reset_default_graph()

In [0]:
graph = tf.Graph()
graph_def = None

# with tf.gfile.GFile(os.path.join('./models', 'deeplabv3_mnv2_dm05_pascal_trainaug.pb'), 'rb') as f:
with tf.gfile.GFile(os.path.join('./models', 'deeplabv3_mnv2_dm05_pascal_trainaug_fp16.pb'), 'rb') as f:
    graph_def = tf.GraphDef.FromString(f.read())
    
if graph_def is None:
    print('Cannot read model.')

In [0]:
with graph.as_default():
    tf.import_graph_def(graph_def, name='')

    sess = tf.Session(graph=graph)
    
    # Read image.
    image = Image.open(os.path.join('./images', 'parrot.jpg'))
        
    width, height = image.size
    resize_ratio = 1.0 * INPUT_SIZE / max(width, height)
    target_size = (int(resize_ratio * width), int(resize_ratio * height))
    resized_image = image.convert('RGB').resize(target_size, Image.ANTIALIAS)
    
    batch_seg_map = sess.run(
        OUTPUT_TENSOR_NAME,
        feed_dict={INPUT_TENSOR_NAME: [np.asarray(resized_image)]})
    seg_map = batch_seg_map[0]
    
    print(seg_map.shape)
    
    vis_segmentation(resized_image, seg_map)